In [2]:
import boto3
import json
import time
import zipfile
import botocore
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.35.18


In [3]:

session = boto3.session.Session()

region = session.region_name

In [4]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
sts_client = session.client('sts')
iam_client = session.client('iam')
s3_client = session.client('s3')
lambda_client = session.client('lambda', region_name='us-east-1')
bedrock_agent_client = session.client('bedrock-agent', region_name='us-east-1')
bedrock_agent_runtime_client = session.client('bedrock-agent-runtime', region_name='us-east-1')

In [6]:
list_buckets_response = s3_client.list_buckets()["Buckets"]
bucket_names = ", ".join(list(map(lambda bucket: bucket["Name"], list_buckets_response)))

print(bucket_names)

ammarindopakwars, b-a-sd1-us-east-1-452982853365, banking-assistant1q11-rt-us-east-1-452982853365, banking-assistant1q11-us-east-1-452982853365, bedrock-docs-kb-agents-saba-us-east-1-452982853365, bedrock-docs-kb-agents-us-east-1-452982853365, booking-agent-sh-us-east-1-452982853365-sh, booking-agent-us-east-1-452982853365, danish-booking-agent-us-east-1-danish, danish-sagemaker-1, ha-banking-assistant-us-east-1-452982853365-lab08, ha-banking-assistant1q11-us-east-1-452982853365, ha-banking-assistant1q11-us-east-1-452982853365-lab08, ha-bedrock-docs-kb-agents-us-east-1-452982853365, ha-booking-agent-us-east-1-452982853365, ha-insurance-claims-agent-us-east-1-452982853365, ha-text-2-sql-agent-lab09-us-east-1-452982853365, ha-text-t-sql-agent-lab09-us-east-1-452982853365, ham-text-t-sql-agent-lab09-us-east-1-452982853365, image-generation-service--serverlessdeploymentbuck-ihc2jso9iyga, image-generation01, insurance-claims-agent-us-east-1-452982853365, irfan-insurance-claims-agent-us-east

In [7]:
account_id = sts_client.get_caller_identity()["Account"]
account_id

'452982853365'

In [8]:
# configuration variables
prefix = "RT"
suffix = f"{prefix}-{region}-{account_id}"
agent_name = f"{prefix}-hr-assistant-func-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'{prefix}-AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = f"anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = f"{prefix}-Agent for providing HR assistance to manage vacation time"
agent_instruction = f"You are an {prefix}-HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = f"{prefix}-VacationsActionGroup"
agent_action_group_description = f"Actions for getting the number of available vactions days for an employee and confirm new time off"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

In [9]:
import sqlite3
import random
from datetime import date, timedelta

In [11]:
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']


In [12]:
for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

     # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year - 3, -1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days - days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y-%m-%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) - date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()




In [13]:
%%writefile RT_lambda_function.py
import os
import json
import shutil
import sqlite3
from datetime import datetime

def get_available_vacations_days(employee_id):
    # Connect to the SQLite database
    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()

    if employee_id:

        # Fetch the available vacation days for the employee
        c.execute("""
            SELECT employee_vacation_days_available
            FROM vacations
            WHERE employee_id = ?
            ORDER BY year DESC
            LIMIT 1
        """, (employee_id,))

        available_vacation_days = c.fetchone()

        if available_vacation_days:
            available_vacation_days = available_vacation_days[0]  # Unpack the tuple
            print(f"Available vacation days for employed_id {employee_id}: {available_vacation_days}")
            conn.close()
            return available_vacation_days
        else:
            return_msg = f"No vacation data found for employed_id {employee_id}"
            print(return_msg)
            return return_msg
            conn.close()
    else:
        raise Exception(f"No employeed id provided")

    # Close the database connection
    conn.close()
    
    
def reserve_vacation_time(employee_id, start_date, end_date):
    # Connect to the SQLite database

    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()
    try:
        # Calculate the number of vacation days
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        vacation_days = (end_date - start_date).days + 1

        # Get the current year
        current_year = start_date.year

        # Check if the employee exists
        c.execute("SELECT * FROM employees WHERE employee_id = ?", (employee_id,))
        employee = c.fetchone()
        if employee is None:
            return_msg = f"Employee with ID {employee_id} does not exist."
            print(return_msg)
            conn.close()
            return return_msg

        # Check if the vacation days are available for the employee in the current year
        c.execute("SELECT employee_vacation_days_available FROM vacations WHERE employee_id = ? AND year = ?", (employee_id, current_year))
        available_days = c.fetchone()
        if available_days is None or available_days[0] < vacation_days:
            return_msg = f"Employee with ID {employee_id} does not have enough vacation days available for the requested period."
            print(return_msg)
            conn.close()
            return return_msg

        # Insert the new vacation into the planned_vacations table
        c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, vacation_days))

        # Update the vacations table with the new vacation days taken
        c.execute("UPDATE vacations SET employee_vacation_days_taken = employee_vacation_days_taken + ?, employee_vacation_days_available = employee_vacation_days_available - ? WHERE employee_id = ? AND year = ?", (vacation_days, vacation_days, employee_id, current_year))

        conn.commit()
        print(f"Vacation saved successfully for employee with ID {employee_id} from {start_date} to {end_date}.")
        # Close the database connection
        conn.close()
        return f"Vacation saved successfully for employee with ID {employee_id} from {start_date} to {end_date}."
    except Exception as e:
        raise Exception(f"Error occurred: {e}")
        conn.rollback()
        # Close the database connection
        conn.close()
        return f"Error occurred: {e}"
    

def lambda_handler(event, context):
    original_db_file = 'employee_database.db'
    target_db_file = '/tmp/employee_database.db'
    if not os.path.exists(target_db_file):
        shutil.copy2(original_db_file, target_db_file)

    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])

    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }

    if function == 'get_available_vacations_days':
        employee_id = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]

        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        vacation_days = get_available_vacations_days(employee_id)
        responseBody =  {
            'TEXT': {
                "body": f"available vacation days for employed_id {employee_id}: {vacation_days}"
            }
        }

    elif function == 'reserve_vacation_time':
        employee_id = None
        start_date = None
        end_date = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]
            if param["name"] == "start_date":
                start_date = param["value"]
            if param["name"] == "end_date":
                end_date = param["value"]
            
        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        if not start_date:
            raise Exception("Missing mandatory parameter: start_date")
        if not end_date:
            raise Exception("Missing mandatory parameter: end_date")
        
        completion_message = reserve_vacation_time(employee_id, start_date, end_date)
        responseBody =  {
            'TEXT': {
                "body": completion_message
            }
        }  



    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse' : {
            'responseBody' :  responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response




Overwriting RT_lambda_function.py


In [14]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName = lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': 'a932995f-b844-4003-a815-fcb3355bb122',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 16 Sep 2024 22:33:35 GMT',
   'x-amzn-requestid': 'a932995f-b844-4003-a815-fcb3355bb122',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [30]:
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("employee_database.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function

lambdaARN = None
try:
    # Try to get the Lambda function
    lambda_function = lambda_client.get_function(FunctionName=lambda_function_name)

    if lambda_function is not None:
        lambdaARN = lambda_function['Configuration']['FunctionArn']
        
    print(f"Lambda function: '{lambda_function_name} having ARN: {lambdaARN}' already exists.")
except botocore.exceptions.ClientError as error:
    # Check if the exception is because the function doesn't exist
    if error.response['Error']['Code'] == 'ResourceNotFoundException':
        print(f"Lambda function '{lambda_function_name}' not found. Creating it now...")
        
        # Create the Lambda function if it does not exist
        lambda_function = lambda_client.create_function(
            FunctionName=lambda_function_name,
            Runtime='python3.12',
            Timeout=180,
            Role=lambda_iam_role['Role']['Arn'],
            Code={'ZipFile': zip_content},
            Handler='lambda_function.lambda_handler'
        )

        lambdaARN = lambda_function['FunctionArn']

        print(f"Lambda function '{lambda_function_name}' created successfully.")
    else:
        # If there's another error, raise it
        raise

Lambda function: 'RT-hr-assistant-func-def-RT-us-east-1-452982853365 having ARN: arn:aws:lambda:us-east-1:452982853365:function:RT-hr-assistant-func-def-RT-us-east-1-452982853365' already exists.


In [16]:
agent_bedrock_policy = None

try:
    # List policies to check if the policy exists
    existing_policies=[]
    hasMorePolicies = True
    marker = ''
    while(hasMorePolicies):
        response = {}
        if len(marker) > 0:
            response = iam_client.list_policies(Scope='Local', Marker=marker)
        else:
            response = iam_client.list_policies(Scope='Local')

        existing_policies.extend(response['Policies'])
        
        # Check if more pages are available
        if not response.get('IsTruncated'):
            hasMorePolicies = False
        else:
            marker = response['Marker']
                   
    for policy in existing_policies:
        if policy['PolicyName'] == agent_bedrock_allow_policy_name:
            agent_bedrock_policy = policy
            break

    
    if agent_bedrock_policy is not None:
        print(f"IAM Policy '{agent_bedrock_allow_policy_name}' already exists with ARN: {agent_bedrock_policy['Arn']}")
    else:
        # If policy does not exist, create it
        print(f"IAM Policy '{agent_bedrock_allow_policy_name}' not found. Creating it now...")
        
        # Create IAM policies for agent
        bedrock_agent_bedrock_allow_policy_statement = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
                    "Effect": "Allow",
                    "Action": "bedrock:InvokeModel",
                    "Resource": [
                        f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
                    ]
                }
            ]
        }

        bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)
        
        agent_bedrock_policy = iam_client.create_policy(
            PolicyName=agent_bedrock_allow_policy_name,
            PolicyDocument=bedrock_policy_json
        )
        

except botocore.exceptions.ClientError as error:
    print(f"An error occurred: {error}")

agent_bedrock_policy

IAM Policy 'RT-hr-assistant-func-def-ba-RT-us-east-1-452982853365' already exists with ARN: arn:aws:iam::452982853365:policy/RT-hr-assistant-func-def-ba-RT-us-east-1-452982853365


{'PolicyName': 'RT-hr-assistant-func-def-ba-RT-us-east-1-452982853365',
 'PolicyId': 'ANPAWS57BZ32RJC52PNW4',
 'Arn': 'arn:aws:iam::452982853365:policy/RT-hr-assistant-func-def-ba-RT-us-east-1-452982853365',
 'Path': '/',
 'DefaultVersionId': 'v2',
 'AttachmentCount': 1,
 'PermissionsBoundaryUsageCount': 0,
 'IsAttachable': True,
 'CreateDate': datetime.datetime(2024, 9, 15, 0, 34, 3, tzinfo=tzutc()),
 'UpdateDate': datetime.datetime(2024, 9, 15, 1, 36, 25, tzinfo=tzutc())}

In [17]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

agent_role = None

try:
    
    agent_role = iam_client.get_role(RoleName=agent_role_name)
    print(f"Agent Role- '{agent_role_name}' already exists.")

except botocore.exceptions.ClientError as error:
    print(f"Agent Role- '{agent_role_name}' NOT found. Creating a new agent Role.")
    assume_role_policy_document_json = json.dumps(assume_role_policy_document)
    agent_role = iam_client.create_role(
        RoleName=agent_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Arn']
)

agent_role

Agent Role- 'RT-AmazonBedrockExecutionRoleForAgents_RT-hr-assistant-func-def' already exists.


{'Role': {'Path': '/',
  'RoleName': 'RT-AmazonBedrockExecutionRoleForAgents_RT-hr-assistant-func-def',
  'RoleId': 'AROAWS57BZ327F3Q7ZQMW',
  'Arn': 'arn:aws:iam::452982853365:role/RT-AmazonBedrockExecutionRoleForAgents_RT-hr-assistant-func-def',
  'CreateDate': datetime.datetime(2024, 9, 15, 0, 34, 47, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]},
  'MaxSessionDuration': 3600,
  'RoleLastUsed': {'LastUsedDate': datetime.datetime(2024, 9, 15, 1, 38, 20, tzinfo=tzutc()),
   'Region': 'us-east-1'}},
 'ResponseMetadata': {'RequestId': '2ae9476a-e0b9-4e7c-a51b-13f5b176f630',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 16 Sep 2024 22:33:45 GMT',
   'x-amzn-requestid': '2ae9476a-e0b9-4e7c-a51b-13f5b176f630',
   'content-type': 'text/xml',
   'content-length': '1030'},
  'RetryAttempts': 0}}

In [23]:

agent_id = None
rtAgent = None
try:

    agents = []
    response = None
    nextToken = 'first'
    while len(nextToken) > 0:

        if nextToken == 'first':
          response   = bedrock_agent_client.list_agents()
        else:
           response   = bedrock_agent_client.list_agents(nextToken=nextToken)

        if 'agentSummaries' in response: 
            agents.extend(response['agentSummaries'])
        
        if 'nextToken' in response and len(response['nextToken']) > 0:
            nextToken = response['nextToken']
        else:
           nextToken = ''
          

    agentSummary = None
    for agent in agents:
       if agent['agentName'] == agent_name:
          agentSummary = agent
          break
          
    
    if agentSummary is not None:   
        print(f"An RT Agent- {agentSummary['agentName']} already exists")

    else:
        rtAgent = bedrock_agent_client.create_agent(
            agentName=agent_name,
            agentResourceRoleArn=agent_role['Role']['Arn'],
            description=agent_description,
            idleSessionTTLInSeconds=1800,
            foundationModel=agent_foundation_model,
            instruction=agent_instruction)

    agent_id = agentSummary['agentId']

except botocore.exceptions.ClientError as Error:
   print(f"An error occurred: {error}")

An RT Agent- RT-hr-assistant-func-def already exists


In [25]:

agent_id

'GDW42A7TH2'

In [26]:
agent_functions = [
    {
        'name' : 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            'employee_id' : {
                'description' : 'the id of the employee to get the available vacations',
                'required': True,
                'type': 'integer'
            }  
        }
    },

    {
        'name' : 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            'employee_id' : {
                'description' : 'the id of the employee to get the available vacations',
                'required': True,
                'type': 'integer'
            },
            'start_date' : {
                'description' : 'the start date for the vacation time',
                'required': True,
                'type': 'string'
            },
            'end_date' : {
                'description' : 'the end date for the vacation time',
                'required': True,
                'type': 'string'
            } 

        }
    },
]

In [63]:
agent_action_group_response = None
rtAgentActionGroupId = None
rtAgentActionGroup = None
rtAgentActionGroupName = None
try:

    actionGroups = []
    response = None
    nextToken = 'first'
    while len(nextToken) > 0:

        if nextToken == 'first':
          response   = bedrock_agent_client.list_agent_action_groups(agentId=agent_id, agentVersion = 'DRAFT')
        else:
           response   = bedrock_agent_client.list_agent_action_groups(agentId=agent_id, agentVersion = 'DRAFT', nextToken=nextToken)

        if 'actionGroupSummaries' in response: 
            actionGroups.extend(response['actionGroupSummaries'])
        
        if 'nextToken' in response and len(response['nextToken']) > 0:
            nextToken = response['nextToken']
        else:
           nextToken = ''
          
    for actionGroup in actionGroups:
       if actionGroup['actionGroupName'] == agent_action_group_name:
          rtAgentActionGroup = actionGroup
          break
          
    
    if rtAgentActionGroup is not None:   
        print(f"An RT Agent- {rtAgentActionGroup['actionGroupName']} already exists")
        rtAgentActionGroupId = rtAgentActionGroup['actionGroupId']

    else:
        agent_action_group_response = bedrock_agent_client.create_agent_action_group(
            agentId = agent_id,
            agentVersion = 'DRAFT',
            actionGroupExecutor = {
                'lambda': lambdaARN
            },
            actionGroupName = agent_action_group_name,
            functionSchema={
                'functions': agent_functions
            },
            description = agent_action_group_description
        )

        rtAgentActionGroup = agent_action_group_response['agentActionGroup']

except botocore.exceptions.ClientError as Error:
   print(f"An error occurred: {error}")
    


An RT Agent- RT-VacationsActionGroup already exists


In [38]:
try:

    statementId= 'allow_bedrock'
    response = lambda_client.get_policy(FunctionName=lambda_function_name)
    policy = json.loads(response['Policy'])

    # Check if the specific permission already exists
    policyExist = False
    for statement in policy['Statement']:
        if statement['Sid'] == statementId:
            print(f"Permission with StatementId '{statementId}' already exists.")
            policyExist = True
            break

    if not policyExist:
        response = lambda_client.add_permission(
                FunctionName= lambda_function_name,
                StatementId= 'allow_bedrock',
                Action='lambda:InvokeFunction',
                Principal='bedrock.amazonaws.com',
                SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
            )
        
        print(f"Permission added: {response}")
        
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'ResourceNotFoundException':
        print(f"No policy found for function {lambda_function_name}. Proceeding to add permission.")
    else:
        # Handle other errors
        print(f"An error occurred: {error}")

Permission with StatementId 'allow_bedrock' already exists.


In [39]:
response = bedrock_agent_client.prepare_agent(agentId = agent_id)

print(response)

{'ResponseMetadata': {'RequestId': '037f80f1-adf0-433d-8ee3-d40597c62987', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 16 Sep 2024 23:17:28 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '037f80f1-adf0-433d-8ee3-d40597c62987', 'x-amz-apigw-id': 'eOJlZFbdoAMEUOA=', 'x-amzn-trace-id': 'Root=1-66e8bc88-78fb6c49165d923b1b311692'}, 'RetryAttempts': 0}, 'agentId': 'GDW42A7TH2', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 9, 16, 23, 17, 28, 764182, tzinfo=tzutc())}


In [41]:
time.sleep(30)

agent_alias_id = 'TSTALIASID'
agent_alias_id

'TSTALIASID'

In [48]:
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText = "How much vacation does employee_id 1 have available?",
    agentId = agent_id,
    agentAliasId= agent_alias_id,
    sessionId = session_id,
    enableTrace = enable_trace,
    endSession=end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-09-17 04:23:10,199] p43536 {651724208.py:14} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 16 Sep 2024 23:22:02 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'a26048f5-7482-11ef-ac4b-f86dcc51c853',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '6921ae51-ab63-4068-ba23-4da2aed7ddf7'},
                      'HTTPStatusCode': 200,
                      'RequestId': '6921ae51-ab63-4068-ba23-4da2aed7ddf7',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x000001D536921A10>,
 'contentType': 'application/json',
 'sessionId': 'a26048f5-7482-11ef-ac4b-f86dcc51c853'}


In [49]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-09-17 04:23:14,918] p43536 {<timed exec>:6} INFO - Final answer ->
Employee with id 1 has 7 vacation days available.


CPU times: total: 15.6 ms
Wall time: 3.05 s


In [47]:
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText = "Please reserve a 1 day vocation for employee id 1 for June 1 2024?",
    agentId = agent_id,
    agentAliasId= agent_alias_id,
    sessionId = session_id,
    enableTrace = enable_trace,
    endSession=end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-09-17 04:22:56,443] p43536 {564666326.py:14} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 16 Sep 2024 23:21:48 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '9ab6c7cb-7482-11ef-a5f4-f86dcc51c853',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'd4a5e20f-ceb7-422b-85aa-53b3cde7b1f4'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd4a5e20f-ceb7-422b-85aa-53b3cde7b1f4',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x000001D5371A9B90>,
 'contentType': 'application/json',
 'sessionId': '9ab6c7cb-7482-11ef-a5f4-f86dcc51c853'}


In [54]:


agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText = "No I want to take last 3 months of the year off as vacation, from Oct 1 2024 to Dec 31 2024",
    agentId = agent_id,
    agentAliasId = agent_alias_id,
    sessionId = session_id,
    enableTrace = enable_trace,
    endSession = end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-09-17 04:36:47,001] p43536 {2086953120.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 16 Sep 2024 23:35:39 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'a26048f5-7482-11ef-ac4b-f86dcc51c853',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'fc59947a-237e-4d6c-bfd8-e3946897c951'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'fc59947a-237e-4d6c-bfd8-e3946897c951',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x000001D537840650>,
 'contentType': 'application/json',
 'sessionId': 'a26048f5-7482-11ef-ac4b-f86dcc51c853'}


In [55]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-09-17 04:37:25,007] p43536 {<timed exec>:6} INFO - Final answer ->
Unfortunately, you do not have enough available vacation days to take the requested 3 month vacation from October 1, 2024 to December 31, 2024. You currently only have 8 vacation days available. Please adjust your vacation request to fit within your available vacation days.


CPU times: total: 0 ns
Wall time: 999 µs


In [60]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId = agent_alias_id,
        sessionId = session_id,
        enableTrace = enable_trace,
        endSession = end_session
    )

    #logger.info(pprint.pprint(agentResponse))

    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)
    



In [61]:
simple_agent_invoke("how much time off does employee 2 have?", agent_id, agent_alias_id, session_id)

[2024-09-17 04:42:14,313] p43536 {3795785164.py:18} INFO - Final answer ->
Employee with id 2 still has 17 vacation days available.


In [65]:
# cleanup

action_group_id = rtAgentActionGroup['actionGroupId']
action_group_name = rtAgentActionGroup['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambdaARN
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [66]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [67]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

{'ResponseMetadata': {'RequestId': '4bd5300a-d6a1-4bde-8f76-cf522ccb6834',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Mon, 16 Sep 2024 23:45:08 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4bd5300a-d6a1-4bde-8f76-cf522ccb6834'},
  'RetryAttempts': 0}}

In [79]:
# Delete IAM Roles and policies

import botocore.exceptions


try:
    for policy in [agent_bedrock_allow_policy_name]:
        iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
except iam_client.exceptions.NoSuchEntityException:
     print(f"RoleName: {agent_role_name}, PolicyArn=arn:aws:iam::{account_id}:policy/{policy}' aleady deleted")



RoleName: RT-AmazonBedrockExecutionRoleForAgents_RT-hr-assistant-func-def, PolicyArn=arn:aws:iam::452982853365:policy/RT-hr-assistant-func-def-ba-RT-us-east-1-452982853365' aleady deleted


In [80]:
try:

    iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
except iam_client.exceptions.NoSuchEntityException:
     print(f"RoleName: {lambda_function_role}, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole' aleady deleted")


RoleName: RT-hr-assistant-func-def-lambda-role-RT-us-east-1-452982853365, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole' aleady deleted


In [82]:
try:
    for role_name in [agent_role_name, lambda_function_role]:
        iam_client.delete_role(
            RoleName=role_name
        )
except iam_client.exceptions.NoSuchEntityException:
     print(f"Agent Role Name: {agent_role_name}, Lambda role name: {lambda_function_role}' aleady deleted")


Agent Role Name: RT-AmazonBedrockExecutionRoleForAgents_RT-hr-assistant-func-def, Lambda role name: RT-hr-assistant-func-def-lambda-role-RT-us-east-1-452982853365' aleady deleted


In [91]:
for policy in [agent_bedrock_policy]:

    try:
        # List policies to check if the policy exists
        policyVersions=[]
        hasMorePolicyVersions = True
        marker = ''
        while(hasMorePolicyVersions):
            response = {}
            if len(marker) > 0:
                response = iam_client.list_policy_versions(PolicyArn=agent_bedrock_policy['Arn'], Marker=marker)
            else:
                response = iam_client.list_policy_versions(PolicyArn=agent_bedrock_policy['Arn'])

            policyVersions.extend(response['Versions'])
            
            # Check if more pages are available
            if not response.get('IsTruncated'):
                hasMorePolicyVersions = False
            else:
                marker = response['Marker']
                    
        for policyVersion in policyVersions:
            if policyVersion['VersionId'] != 'v2':
                iam_client.delete_policy_version(PolicyArn=agent_bedrock_policy['Arn'], VersionId=policyVersion['VersionId'])
        
        iam_client.delete_policy(PolicyArn=agent_bedrock_policy['Arn'])

    except Exception as e:
     print(f"Exception: {e}")
   


    